In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
#weather_dir = 'I:/Weather Data/'
weather_dir = '.'

In [44]:
all_stations_df = pd.read_fwf(f'{weather_dir}/allstations.txt', header=None)
all_stations_df.columns = ['ID', 'Latitude','Longitude','Elevation','State','Name','GSNFLAG','HCNFLAG','WMOID']
all_stations_df.head()

,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID
0,AQC00914000,-14.3167,-170.7667,408.4,AS,AASUFOU,NaN,NaN,NaN
1,AQW00061705,-14.3306,-170.7136,3.7,AS,PAGO PAGO WSO AP,NaN,GSN,91765.0
2,CAW00064757,44.2325,-79.7811,246.0,ON,EGBERT 1 W,NaN,NaN,NaN
3,CQC00914080,15.2136,145.7497,252.1,MP,CAPITOL HILL 1,NaN,NaN,NaN
4,CQC00914801,14.1717,145.2428,179.2,MP,ROTA AP,NaN,NaN,91221.0


In [46]:
daily_tmin_df = pd.read_fwf(f'{weather_dir}/dly-tmin-normal.txt', header=None)
daily_tmin_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmin_df = pd.melt(daily_tmin_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmin_df.columns = ['STNID','MONTH','DAY','VALUE']
daily_tmin_df = daily_tmin_df[daily_tmin_df['VALUE'] != '-8888']
daily_tmin_df['FLAG'] = daily_tmin_df['VALUE'].map(lambda x: x[-1])
daily_tmin_df['TMIN'] = daily_tmin_df['VALUE'].map(lambda x: int(x[0:-1])/10)

tmin_df = daily_tmin_df.groupby('STNID')['TMIN'].min()
tmin_df

STNID
AQW00061705    76.2
CAW00064757    10.1
CQC00914080    72.9
CQC00914801    73.2
FMC00914395    72.1
               ... 
VQC00671740    71.5
VQC00671980    70.1
VQW00011624    72.2
VQW00011640    72.4
WQW00041606    73.2
Name: TMIN, Length: 7501, dtype: float64

In [45]:
daily_tmax_df = pd.read_fwf(f'{weather_dir}/dly-tmax-normal.txt', header=None)
daily_tmax_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmax_df = pd.melt(daily_tmax_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmax_df.columns = ['STNID','MONTH','DAY','VALUE']
daily_tmax_df = daily_tmax_df[daily_tmax_df['VALUE'] != '-8888']
daily_tmax_df['FLAG'] = daily_tmax_df['VALUE'].map(lambda x: x[-1])
daily_tmax_df['TMAX'] = daily_tmax_df['VALUE'].map(lambda x: int(x[0:-1])/10)

tmax_df = daily_tmax_df.groupby('STNID')['TMAX'].max()
tmax_df

STNID
AQW00061705    88.2
CAW00064757    79.6
CQC00914080    85.1
CQC00914801    87.4
FMC00914395    89.0
               ... 
VQC00671740    88.1
VQC00671980    88.2
VQW00011624    89.4
VQW00011640    90.3
WQW00041606    89.0
Name: TMAX, Length: 7501, dtype: float64

In [47]:
days_under_32 = pd.read_fwf('ann-tmin-avgnds-lsth032.txt', header=None)
days_under_32.columns = ['STNID','VALUE']
days_under_32['FLAG'] = days_under_32['VALUE'].map(lambda x: x[-1])
days_under_32['DAYS_LT_32'] = days_under_32['VALUE'].map(lambda x: int(x[0:-1])/10)

days_under_32 = days_under_32.set_index('STNID')['DAYS_LT_32']
days_under_32

STNID
AQW00061705      0.0
CAW00064757    153.2
CQC00914080      0.0
CQC00914801      0.0
FMC00914395      0.0
               ...  
VQC00671740      0.0
VQC00671980      0.0
VQW00011624      0.0
VQW00011640      0.0
WQW00041606      0.0
Name: DAYS_LT_32, Length: 7501, dtype: float64

In [48]:
days_over_80 = pd.read_fwf('ann-tmax-avgnds-grth080.txt', header=None)
days_over_80.columns = ['STNID','VALUE']
days_over_80['FLAG'] = days_over_80['VALUE'].map(lambda x: x[-1])
days_over_80['DAYS_GT_80'] = days_over_80['VALUE'].map(lambda x: int(x[0:-1])/10)

days_over_80 = days_over_80.set_index('STNID')['DAYS_GT_80']
days_over_80

STNID
AQW00061705    361.4
CAW00064757     35.2
CQC00914080    315.2
CQC00914801    361.4
FMC00914395    364.8
               ...  
VQC00671740    356.9
VQC00671980    354.9
VQW00011624    362.5
VQW00011640    362.0
WQW00041606    356.6
Name: DAYS_GT_80, Length: 7501, dtype: float64

In [69]:
precip_df = pd.read_fwf(f'{weather_dir}/ann-prcp-normal.txt', header=None)
precip_df.columns = ['STNID','VALUE']
precip_df['FLAG'] = precip_df['VALUE'].map(lambda x: x[-1])
precip_df['ANN_PRECIP'] = precip_df['VALUE'].map(lambda x: int(x[0:-1])/100)

precip_df = precip_df.set_index('STNID')['ANN_PRECIP']
precip_df

STNID
AQC00914000    213.92
AQW00061705    122.63
CAW00064757     31.72
CQC00914080     83.39
CQC00914801     91.24
                ...  
VQC00677600     43.84
VQC00679450     43.70
VQW00011624     38.67
VQW00011640     39.34
WQW00041606     34.46
Name: annual_precipitation, Length: 9307, dtype: float64

In [70]:
station_data_df = pd.merge(all_stations_df[['ID','Latitude','Longitude']], pd.DataFrame({'TMIN':tmin_df, 'TMAX':tmax_df,'DAYS_LT_32':days_under_32, 'DAYS_GT_80':days_over_80, 'ANN_PRECIP':precip_df}), left_on='ID', right_index=True)
station_data_df

,ID,Latitude,Longitude,TMIN,TMAX,DAYS_LT_32,DAYS_GT_80,ANN_PRECIP
0,AQC00914000,-14.3167,-170.7667,NaN,NaN,NaN,NaN,213.92
1,AQW00061705,-14.3306,-170.7136,76.2,88.2,0.0,361.4,122.63
2,CAW00064757,44.2325,-79.7811,10.1,79.6,153.2,35.2,31.72
3,CQC00914080,15.2136,145.7497,72.9,85.1,0.0,315.2,83.39
4,CQC00914801,14.1717,145.2428,73.2,87.4,0.0,361.4,91.24
5,CQC00914855,15.1189,145.7294,NaN,NaN,NaN,NaN,69.76
6,CQC00914874,15.0000,145.6333,NaN,NaN,NaN,NaN,78.68
7,FMC00914213,9.4508,138.0620,NaN,NaN,NaN,NaN,115.94
8,FMC00914325,1.0833,154.8000,NaN,NaN,NaN,NaN,128.40
9,FMC00914395,5.3544,162.9533,72.1,89.0,0.0,364.8,193.34


In [61]:
lat_longs = pd.read_csv('lat_longs_to_fips.csv', dtype={'FIPS_COUNTY':str}).drop_duplicates()

In [63]:
pd.merge(station_data_df, lat_longs, on=['Latitude','Longitude'])

,ID,Latitude,Longitude,TMIN,TMAX,DAYS_LT_32,DAYS_GT_80,FIPS_COUNTY
0,USC00010063,34.2553,-87.1814,28.5,90.1,66.7,137.4,01133
1,USC00010160,32.9453,-85.9481,31.4,90.9,57.3,155.6,01123
2,USC00010178,33.1272,-88.1550,31.0,92.7,61.4,172.5,01107
3,USC00010252,31.3072,-86.5225,31.9,91.8,62.9,186.4,01039
4,USC00010260,34.9092,-87.2747,26.4,89.8,79.5,134.7,01077
5,USC00010369,33.2942,-85.7789,29.4,89.9,68.2,149.4,01027
6,USC00010390,34.7753,-86.9508,29.8,90.2,62.1,143.3,01083
7,USC00010402,31.1819,-87.4389,36.9,91.3,30.3,177.0,01053
8,USC00010425,32.5992,-85.4653,32.2,90.5,42.5,154.8,01081
9,USC00010430,32.6000,-85.5000,33.5,91.1,39.8,158.5,01081


In [103]:
weather_data_by_county = pd.merge(station_data_df, lat_longs, on=['Latitude','Longitude']).groupby('FIPS_COUNTY').agg({
        'Latitude':'mean',
        'Longitude':'mean',
        'TMIN':'min', 
        'TMAX':'max', 
        'DAYS_LT_32':'mean', 
        'DAYS_GT_80':'mean',
        'ANN_PRECIP':'max'
    })
weather_data_by_county['ANN_PRECIP'] = weather_data_by_county['ANN_PRECIP'].fillna(-1)
weather_data_by_county

,Latitude,Longitude,TMIN,TMAX,DAYS_LT_32,DAYS_GT_80,ANN_PRECIP
FIPS_COUNTY,,,,,,,
01003,30.636125,-87.810900,36.8,91.2,25.100000,174.075000,68.90
01005,31.944433,-85.222967,31.8,93.4,42.833333,180.066667,50.14
01007,32.866100,-87.238300,30.3,90.9,67.300000,158.800000,57.23
01009,33.947800,-86.469200,29.9,90.8,66.600000,151.200000,55.41
01011,32.014200,-85.746400,32.9,90.6,55.900000,164.500000,55.42
01013,31.790000,-86.608600,33.1,90.6,49.600000,165.100000,57.36
01015,33.587200,-85.855600,32.7,90.1,48.000000,149.700000,49.67
01017,32.888150,-85.311400,29.1,90.9,60.900000,147.350000,54.55
01019,34.150000,-85.684700,27.7,90.5,78.100000,146.300000,54.10


In [104]:
avg_weekly_wage = pd.read_csv('avg_weekly_wage.csv', dtype={'FIPS_COUNTY':str})
avg_weekly_wage.head()

,FIPS_COUNTY,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01001,"Autauga County, Alabama",802
1,01003,"Baldwin County, Alabama",717
2,01005,"Barbour County, Alabama",727
3,01007,"Bibb County, Alabama",785
4,01009,"Blount County, Alabama",683


In [105]:
sunshine_df = pd.read_csv('sunlight.txt', delimiter='\t', header=None, dtype={2:str})
sunshine_df.columns = ['Notes','County','FIPS_COUNTY','AVG_DLY_SUN']
sunshine_df = sunshine_df.set_index('FIPS_COUNTY')['AVG_DLY_SUN']
sunshine_df['12086'] = sunshine_df['12025'] # Set Miami-Dade data to Dade county data
sunshine_df

FIPS_COUNTY
01001    17103.72
01003    17660.59
01005    17315.13
01007    16904.87
01009    16393.63
           ...   
56039    16531.32
56041    17123.76
56043    16171.03
56045    15923.65
12086    19744.21
Name: AVG_DLY_SUN, Length: 3112, dtype: float64

In [106]:
data_df = weather_data_by_county.copy()

In [107]:
data_df['AVG_DLY_SUN'] = sunshine_df
data_df = pd.merge(data_df, avg_weekly_wage, left_index=True, right_on='FIPS_COUNTY')

In [108]:
data_df[data_df.isnull().any(axis=1)]

,Latitude,Longitude,TMIN,TMAX,DAYS_LT_32,DAYS_GT_80,ANN_PRECIP,AVG_DLY_SUN,FIPS_COUNTY,COUNTY_NAME,AVG_WEEKLY_WAGE
68,55.268750,-161.624600,23.2,58.3,118.250000,0.000000,41.67,NaN,02013,"Aleutians East Borough, Alaska",1100
69,54.450060,-101.776100,19.9,59.0,133.020000,155.540000,61.63,NaN,02016,"Aleutians West Census Area, Alaska",1111
70,61.161306,-149.555367,1.8,69.7,195.161111,1.022222,156.00,NaN,02020,"Anchorage Municipality, Alaska",1159
71,60.785000,-161.829200,0.3,64.2,214.300000,0.800000,18.54,NaN,02050,"Bethel Census Area, Alaska",879
72,58.679400,-156.629400,8.6,64.0,192.000000,0.700000,19.49,NaN,02060,"Bristol Bay Borough, Alaska",966
73,63.662833,-149.244733,-12.3,74.2,238.133333,3.683333,16.53,NaN,02068,"Denali Borough, Alaska",1297
74,59.050000,-158.516700,9.2,61.4,191.000000,0.000000,25.32,NaN,02070,"Dillingham Census Area, Alaska",898
75,64.867495,-147.598327,-17.1,77.4,231.377273,9.540909,15.83,NaN,02090,"Fairbanks North Star Borough, Alaska",1001
76,59.240700,-135.479400,17.1,65.7,145.300000,2.050000,66.44,NaN,02100,"Haines Borough, Alaska",1049
77,58.229017,-135.745150,21.7,66.1,115.716667,0.500000,150.71,NaN,02105,"Hoonah-Angoon Census Area, Alaska",650


In [109]:
data_df.dtypes

Latitude           float64
Longitude          float64
TMIN               float64
TMAX               float64
DAYS_LT_32         float64
DAYS_GT_80         float64
ANN_PRECIP         float64
AVG_DLY_SUN        float64
FIPS_COUNTY         object
COUNTY_NAME         object
AVG_WEEKLY_WAGE      int64
dtype: object

In [110]:
data_df.to_csv('data.csv')

In [112]:
data_df.describe()

,Latitude,Longitude,TMIN,TMAX,DAYS_LT_32,DAYS_GT_80,ANN_PRECIP,AVG_DLY_SUN,AVG_WEEKLY_WAGE
count,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2833.000000,2865.000000
mean,38.619976,-92.875799,19.943455,88.544852,108.902900,109.356765,39.367613,16043.780487,816.282024
std,5.376906,12.983887,11.645269,5.574722,53.311443,50.064184,17.640194,1528.363603,217.973413
min,19.636305,-166.769600,-30.300000,58.300000,0.000000,0.000000,-1.000000,12945.790000,443.000000
25%,34.814375,-98.914875,11.900000,84.800000,67.050000,71.333333,27.740000,14764.310000,689.000000
50%,38.615800,-91.260550,20.200000,88.900000,112.550000,105.000000,40.870000,15851.130000,768.000000
75%,42.111700,-83.787200,28.300000,92.200000,151.300000,142.950000,49.040000,17179.190000,878.000000
max,70.085811,-67.513089,59.900000,117.000000,279.811111,409.300000,240.020000,20774.770000,3153.000000


In [3]:
daily_tmin_df = pd.read_fwf(f'{weather_dir}/dly-tmin-normal.txt', header=None)
daily_tmin_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmin_df = pd.melt(daily_tmin_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmin_df.columns = ['STNID','MONTH','DAY','VALUE']

daily_tmin_df = daily_tmin_df[daily_tmin_df['VALUE'] != '-8888']
daily_tmin_df['FLAG'] = daily_tmin_df['VALUE'].map(lambda x: x[-1])
daily_tmin_df['VALUE'] = daily_tmin_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmin_df

,STNID,MONTH,DAY,VALUE,FLAG
0,AQW00061705,1,1,77.2,C
1,AQW00061705,2,1,77.4,C
2,AQW00061705,3,1,77.6,C
3,AQW00061705,4,1,77.4,C
4,AQW00061705,5,1,77.3,C
5,AQW00061705,6,1,77.2,C
6,AQW00061705,7,1,76.5,C
7,AQW00061705,8,1,76.2,C
8,AQW00061705,9,1,76.5,C
9,AQW00061705,10,1,76.8,C


In [4]:
daily_tmax_df = pd.read_fwf(f'{weather_dir}/dly-tmax-normal.txt', header=None)
daily_tmax_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmax_df = pd.melt(daily_tmax_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmax_df.columns = ['STNID','MONTH','DAY','VALUE']

daily_tmax_df = daily_tmax_df[daily_tmax_df['VALUE'] != '-8888']
daily_tmax_df['FLAG'] = daily_tmax_df['VALUE'].map(lambda x: x[-1])
daily_tmax_df['VALUE'] = daily_tmax_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmax_df

,STNID,MONTH,DAY,VALUE,FLAG
0,AQW00061705,1,1,87.5,C
1,AQW00061705,2,1,87.8,C
2,AQW00061705,3,1,88.2,C
3,AQW00061705,4,1,87.9,C
4,AQW00061705,5,1,86.9,C
5,AQW00061705,6,1,85.5,C
6,AQW00061705,7,1,84.6,C
7,AQW00061705,8,1,84.0,C
8,AQW00061705,9,1,84.9,C
9,AQW00061705,10,1,85.6,C


In [5]:
daily_temp_df = pd.merge(daily_tmin_df, daily_tmax_df, on=['STNID','MONTH','DAY'], suffixes=['_MIN','_MAX'])
daily_temp_df

,STNID,MONTH,DAY,VALUE_MIN,FLAG_MIN,VALUE_MAX,FLAG_MAX
0,AQW00061705,1,1,77.2,C,87.5,C
1,AQW00061705,2,1,77.4,C,87.8,C
2,AQW00061705,3,1,77.6,C,88.2,C
3,AQW00061705,4,1,77.4,C,87.9,C
4,AQW00061705,5,1,77.3,C,86.9,C
5,AQW00061705,6,1,77.2,C,85.5,C
6,AQW00061705,7,1,76.5,C,84.6,C
7,AQW00061705,8,1,76.2,C,84.0,C
8,AQW00061705,9,1,76.5,C,84.9,C
9,AQW00061705,10,1,76.8,C,85.6,C


In [6]:
all_stations_df = pd.read_fwf(f'{weather_dir}/allstations.txt', header=None)
all_stations_df.columns = ['ID', 'Latitude','Longitude','Elevation','State','Name','GSNFLAG','HCNFLAG','WMOID']
all_stations_df.head()

,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID
0,AQC00914000,-14.3167,-170.7667,408.4,AS,AASUFOU,NaN,NaN,NaN
1,AQW00061705,-14.3306,-170.7136,3.7,AS,PAGO PAGO WSO AP,NaN,GSN,91765.0
2,CAW00064757,44.2325,-79.7811,246.0,ON,EGBERT 1 W,NaN,NaN,NaN
3,CQC00914080,15.2136,145.7497,252.1,MP,CAPITOL HILL 1,NaN,NaN,NaN
4,CQC00914801,14.1717,145.2428,179.2,MP,ROTA AP,NaN,NaN,91221.0


In [7]:
# lat_longs = list(stations.apply(lambda row: {'Latitude':row['Latitude'],'Longitude':row['Longitude']}, axis='columns'))
# lat_longs

# for lat_long in lat_longs:
#     url = f"https://geo.fcc.gov/api/census/block/find?latitude={lat_long['Latitude']}&longitude={lat_long['Longitude']}&showall=false&format=json"
#     response = requests.get(url)
#     if response.ok:
#         lat_long['FIPS_COUNTY'] = response.json()['County']['FIPS']
#     print(lat_long)

# pd.DataFrame(lat_longs).to_csv('lat_longs_to_fips.csv')

lat_longs = pd.read_csv('lat_longs_to_fips.csv', dtype={'FIPS_COUNTY':str})
lat_longs

,Latitude,Longitude,FIPS_COUNTY
0,34.2553,-87.1814,01133
1,32.9453,-85.9481,01123
2,33.1272,-88.1550,01107
3,31.3072,-86.5225,01039
4,34.9092,-87.2747,01077
5,33.2942,-85.7789,01027
6,34.7753,-86.9508,01083
7,31.1819,-87.4389,01053
8,32.5992,-85.4653,01081
9,32.6000,-85.5000,01081


In [8]:
daily_temp_df = pd.merge(daily_temp_df, all_stations_df, left_on='STNID', right_on='ID')
daily_temp_df = pd.merge(daily_temp_df, lat_longs, on=['Latitude','Longitude'])
daily_temp_df

,STNID,MONTH,DAY,VALUE_MIN,FLAG_MIN,VALUE_MAX,FLAG_MAX,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID,FIPS_COUNTY
0,USC00010063,1,1,29.1,Q,49.6,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
1,USC00010063,2,1,30.3,Q,51.8,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
2,USC00010063,3,1,36.1,Q,58.1,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
3,USC00010063,4,1,44.0,Q,68.0,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
4,USC00010063,5,1,52.1,Q,74.9,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
5,USC00010063,6,1,61.0,Q,83.1,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
6,USC00010063,7,1,67.1,Q,87.2,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
7,USC00010063,8,1,68.2,Q,89.9,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
8,USC00010063,9,1,65.0,Q,87.6,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133
9,USC00010063,10,1,54.0,Q,77.9,Q,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133


In [10]:
daily_min_temp_by_county = daily_temp_df.groupby(['FIPS_COUNTY','MONTH','DAY'])['VALUE_MIN'].min()
daily_max_temp_by_county = daily_temp_df.groupby(['FIPS_COUNTY','MONTH','DAY'])['VALUE_MAX'].max()
lat_long_by_county = daily_temp_df.groupby(['FIPS_COUNTY'])[['Latitude','Longitude']].mean()
daily_temp_by_county = pd.merge(daily_min_temp_by_county, daily_max_temp_by_county, left_index=True, right_index=True)
daily_temp_by_county = pd.merge(daily_temp_by_county, lat_long_by_county, left_index=True, right_index=True)
daily_temp_by_county

VALUE_MIN  VALUE_MAX   Latitude  Longitude
FIPS_COUNTY MONTH DAY                                            
01003       1     1         37.6       61.3  30.636125   -87.8109
                  2         37.5       61.3  30.636125   -87.8109
                  3         37.4       61.2  30.636125   -87.8109
                  4         37.3       61.2  30.636125   -87.8109
                  5         37.2       61.2  30.636125   -87.8109
...                          ...        ...        ...        ...
56045       12    27         7.4       35.1  43.927500  -104.5146
                  28         7.4       35.2  43.927500  -104.5146
                  29         7.4       35.3  43.927500  -104.5146
                  30         7.5       35.4  43.927500  -104.5146
                  31         7.5       35.5  43.927500  -104.5146

[1049322 rows x 4 columns]

In [11]:
precip_df = pd.read_fwf(f'{weather_dir}/ann-prcp-normal.txt', header=None)
precip_df.columns = ['STNID','VALUE']
precip_df['FLAG'] = precip_df['VALUE'].map(lambda x: x[-1])
precip_df['annual_precipitation'] = precip_df['VALUE'].map(lambda x: int(x[0:-1])/100)

precip_df = pd.merge(precip_df, all_stations_df, left_on='STNID', right_on='ID')
precip_df = pd.merge(precip_df, lat_longs, on=['Latitude','Longitude'])

precip_by_county = precip_df.groupby('FIPS_COUNTY')['annual_precipitation'].max()
#precip_by_county['13193']
#precip_df[precip_df['FIPS_COUNTY'] == '13193']
precip_df[precip_df['FIPS_COUNTY'] == '01003']

,STNID,VALUE,FLAG,annual_precipitation,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID,FIPS_COUNTY
11,USC00010583,6890S,S,68.90,USC00010583,30.8839,-87.7853,82.6,AL,BAY MINETTE,NaN,NaN,NaN,01003
28,USC00012813,6787S,S,67.87,USC00012813,30.5467,-87.8808,7.0,AL,FAIRHOPE 2 NE,NaN,NaN,NaN,01003
60,USC00016988,6732R,R,67.32,USC00016988,30.5653,-87.7017,49.1,AL,ROBERTSDALE,NaN,NaN,NaN,01003


In [12]:
df = pd.DataFrame({
    'min_temp': daily_temp_by_county.groupby('FIPS_COUNTY')['VALUE_MIN'].min(), 
    'max_temp':daily_temp_by_county.groupby('FIPS_COUNTY')['VALUE_MAX'].max(),
    'latitude':daily_temp_by_county.groupby('FIPS_COUNTY')['Latitude'].mean(),
    'longitude':daily_temp_by_county.groupby('FIPS_COUNTY')['Longitude'].mean(),
    'annual_precipitation':precip_by_county
}).reset_index()
df.rename({'index':'FIPS_COUNTY'}, axis='columns', inplace=True)
df['annual_precipitation'].isna().sum()

46

In [13]:
avg_weekly_wage = pd.read_csv('avg_weekly_wage.csv', dtype={'FIPS_COUNTY':str})
avg_weekly_wage.head()

,FIPS_COUNTY,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01001,"Autauga County, Alabama",802
1,01003,"Baldwin County, Alabama",717
2,01005,"Barbour County, Alabama",727
3,01007,"Bibb County, Alabama",785
4,01009,"Blount County, Alabama",683


In [14]:
data_df = pd.merge(df, avg_weekly_wage, on='FIPS_COUNTY')
data_df

,FIPS_COUNTY,min_temp,max_temp,latitude,longitude,annual_precipitation,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01003,36.8,91.2,30.636125,-87.810900,68.90,"Baldwin County, Alabama",717
1,01005,31.8,93.4,31.944433,-85.222967,50.14,"Barbour County, Alabama",727
2,01007,30.3,90.9,32.866100,-87.238300,57.23,"Bibb County, Alabama",785
3,01009,29.9,90.8,33.947800,-86.469200,55.41,"Blount County, Alabama",683
4,01011,32.9,90.6,32.014200,-85.746400,55.42,"Bullock County, Alabama",673
5,01013,33.1,90.6,31.790000,-86.608600,57.36,"Butler County, Alabama",659
6,01015,32.7,90.1,33.587200,-85.855600,49.67,"Calhoun County, Alabama",794
7,01017,29.1,90.9,32.888150,-85.311400,54.55,"Chambers County, Alabama",745
8,01019,27.7,90.5,34.150000,-85.684700,54.10,"Cherokee County, Alabama",658
9,01021,31.9,91.0,32.864200,-86.644800,57.88,"Chilton County, Alabama",741


In [28]:
sunshine_df = pd.read_csv('sunlight.txt', delimiter='\t', header=None, dtype={2:str})
sunshine_df.columns = ['Notes','County','FIPS_COUNTY','avg_daily_sunlight']
sunshine_df.drop(['Notes','County'], axis=1, inplace=True)
sunshine_df

,FIPS_COUNTY,avg_daily_sunlight
0,01001,17103.72
1,01003,17660.59
2,01005,17315.13
3,01007,16904.87
4,01009,16393.63
5,01011,17296.02
6,01013,17323.69
7,01015,16447.09
8,01017,16997.01
9,01019,16331.42


In [29]:
data_df = pd.merge(data_df, sunshine_df, on='FIPS_COUNTY')

In [32]:
days_under_32 = pd.read_fwf('ann-tmin-avgnds-lsth032.txt')
days_under_32

,AQW00061705,0C
0,CAW00064757,1532Q
1,CQC00914080,0P
2,CQC00914801,0R
3,FMC00914395,0P
4,FMC00914419,0P
5,FMC00914446,0R
6,FMC00914482,0P
7,FMC00914720,0P
8,FMC00914761,0P
9,FMC00914831,0R


In [30]:
data_df['annual_precipitation'] = data_df['annual_precipitation'].fillna(-1)
data_df.to_csv('data.csv')

In [31]:
data_df.head()

,FIPS_COUNTY,min_temp,max_temp,latitude,longitude,annual_precipitation,COUNTY_NAME,AVG_WEEKLY_WAGE,avg_daily_sunlight
0,01003,36.8,91.2,30.636125,-87.810900,68.90,"Baldwin County, Alabama",717,17660.59
1,01005,31.8,93.4,31.944433,-85.222967,50.14,"Barbour County, Alabama",727,17315.13
2,01007,30.3,90.9,32.866100,-87.238300,57.23,"Bibb County, Alabama",785,16904.87
3,01009,29.9,90.8,33.947800,-86.469200,55.41,"Blount County, Alabama",683,16393.63
4,01011,32.9,90.6,32.014200,-85.746400,55.42,"Bullock County, Alabama",673,17296.02
